# Style transfer

#### Setup

In [1]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change the directory
    os.chdir('/content/drive/MyDrive/ColabNotebooks/style-transfer')
    is_colab = True
    print("Running on Colab")

except:
    is_colab = False
    print("Running locally")

Running locally


In [5]:
# 35s
if is_colab:
    ! pip install pretty_midi
    # ! apt-get install musescore
    ! pip install dfply

    # ! pip install pyfluidsynth
    # ! apt install fluidsynth

    # !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [5]:
#3s
import random
random.seed(42)

import glob
import pandas as pd
from utils.files_utils import save_pickle, load_pickle, data_path, preprocessed_data_path, datasets_path
from utils.plots_utils import plot_train
from utils.audio_management import save_audios
from model.train import train_model
from model.colab_tension_vae.params import init
from pipeline_tests.test_training import test_reconstruction
from preprocessing import preprocess_data
from tensorflow import keras

## Parámetros

In [12]:
#@title Parámetros ajustables

#@markdown Seleccionar directorios de los datasets
dataset_name = "bach-rag-moz-fres-4" #@param {type:"string"}
model_name = "brmf_4b" #@param {type:"string"}
config_name = "4bar" #@param ["8bar", "4bar"]
init(config_name)

#@markdown ¿Entrenamos un nuevo modelo?
if_train = False #@param {type:"boolean"}
verbose = True #@param {type:"boolean"}

#@markdown ¿Cuántas épocas? ¿Cada cuánto grabamos?
epochs =  10#@param {type:"integer"}
checkpt =  1#@param {type:"integer"}

#@markdown Datasets:
dataset1 = "Bach/" #@param {type:"string"}
dataset2 = "ragtime/" #@param {type:"string"}
dataset3 = "Mozart/" #@param {type:"string"}
dataset4 = "Frescobaldi/" #@param {type:"string"}

ds1_path = os.path.join(datasets_path, dataset1)
ds2_path = os.path.join(datasets_path, dataset2)
ds3_path = os.path.join(datasets_path, dataset3)
ds4_path = os.path.join(datasets_path, dataset4)

songs = {
    dataset1[:-1]: [os.path.join(ds1_path, path) for path in os.listdir(ds1_path)],
    dataset2[:-1]: [os.path.join(ds2_path, path) for path in os.listdir(ds2_path)],
    dataset3[:-1]: [os.path.join(ds3_path, path) for path in os.listdir(ds3_path)],
    dataset4[:-1]: [os.path.join(ds4_path, path) for path in os.listdir(ds4_path)],
}

In [ ]:
#@title Transformar estilo
ds_original = "Bach" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]
ds_objetivo = "Mozart" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]

nombre_pickle = ds_original+"2"+ds_objetivo

____
# Preprocessing and Model

Se lo lleva al formato lista de:

- $n$ canciones = lista de:
- 2 voces = lista de:
- $e$ eventos (notas, silencios, etc.)


Armamos el DataFrame con el que manejamos las canciones. Cada fila es una canción y las columnas son:
- `Autor`
- `Título`
- `Id roll`: qué matriz es de la canción
- `Roll` objeto (GuoRoll) con:
    - `matrix`: matriz de $16*n \times 89$ con $n=$ la cantidad de compases
    - `bars`: cantidad de compases por matriz (es el mismo para todo el dataset, con lo cual, podría eliminarse la redundancia en un trabajo futuro)
    - `song`: referencia al objeto `song` al que pertenece si fue obtenido de ahí. Si fue obtenido desde un embedding, es `None` (en un trabajo futuro podría cambiárselo por un singleton).
    - `score`: partitura obtenida a partir de la matriz
    - `midi`: Pretty MIDI obtenido a partir de la matriz

Luego le agregaremos:
- `Embedding`: el vector resultado de encodear el roll

## Preprocessing

In [13]:
try:
    df_preprocessed = load_pickle(file_name=preprocessed_data_path+dataset_name, verbose=verbose)
except:
    print("Processing dataset")
    df_preprocessed = preprocess_data(songs)
    save_pickle(df_preprocessed, file_name=preprocessed_data_path+dataset_name, verbose=verbose)

Loaded file: <_io.BufferedReader name='/home/urania/Documentos/Tesis/src/style-transfer/data/preprocessed_data/bach-rag-moz-fres-4.pkl'>


## Model

In [ ]:
if if_train:
    vae = train_model(
        df=df_preprocessed,
        model_name=model_name,
        final_epoch=epochs,
        ckpt=checkpt,
        verbose=verbose
    )
else:
    vae = keras.models.load_model(data_path + f"saved_models/{model_name}/")

In [ ]:
glob.glob(rf"{data_path}logs/{model_name}_*.csv")

In [ ]:
id_file =  790#@param {type:"integer"}

callbacks = pd.read_csv(data_path + f"logs/{model_name}_{id_file}.csv")
plot_train(callbacks, epochs)

## Test reconstruction

In [ ]:
if is_colab:
  ! sudo apt install lilypond

test_reconstruction(df_preprocessed, vae, model_name, samples=5, inplace=False, verbose=verbose)

# Working space

In [13]:
def save_midis(df, verbose=False):
    rolls_generated = df[df.columns[-1]]
    midis = [r.midi for r in rolls_generated]
    save_audios(df['Titulo'], midis, verbose=verbose)


In [14]:
df_preprocessed.head()

,Autor,Titulo,roll
0,Bach,bwv0401,<roll.guoroll.GuoRoll object at 0x7f1b20e39340>
1,Bach,bwv0401,<roll.guoroll.GuoRoll object at 0x7f1b208fab50>
2,Bach,bwv0401,<roll.guoroll.GuoRoll object at 0x7f1b20b54970>
3,Bach,bwv0381,<roll.guoroll.GuoRoll object at 0x7f1b20b50b20>
4,Bach,bwv0381,<roll.guoroll.GuoRoll object at 0x7f1b20a344f0>


In [4]:
from utils.files_utils import get_transferred_path

df_reconstructed = load_pickle(get_transferred_path('Bach', 'ragtime', "brmf_4b"))
df_reconstructed.head()

,Autor,Titulo,roll,Embedding,Embedding-NewRoll
1866,ragtime,antoinette.mid,<roll.guoroll.GuoRollSmall object at 0x7f0cd97...,"[-0.0043747313, 1.41698, -1.5279223, -0.020506...",<roll.guoroll.GuoRoll object at 0x7f0cd8b51190>
758,ragtime,breeze.mid,<roll.guoroll.GuoRollSmall object at 0x7f0cd93...,"[-0.051136367, 0.73456746, -0.91220284, -2.043...",<roll.guoroll.GuoRoll object at 0x7f0cd8b80220>
70,Bach,bwv0253.mid,<roll.guoroll.GuoRollSmall object at 0x7f0cd93...,"[-0.6771048, -0.15007068, -0.12760165, 0.50512...",<roll.guoroll.GuoRoll object at 0x7f0cd8b39190>
29,Bach,bwv0254.mid,<roll.guoroll.GuoRollSmall object at 0x7f0cd93...,"[-2.185468, 1.9231098, 1.0495294, 5.617977, 2....",<roll.guoroll.GuoRoll object at 0x7f0cd8add670>
91,Bach,bwv0255.mid,<roll.guoroll.GuoRollSmall object at 0x7f0cd93...,"[-0.8813409, -0.5599785, -1.5920982, 3.2512496...",<roll.guoroll.GuoRoll object at 0x7f0cd8aff6a0>


In [4]:
df_transferred = load_pickle(get_transferred_path('Bach', 'ragtime', "brmf_4b"))
df_transferred.head()

,Autor,Titulo,roll,Embedding,Mutacion_add,Mutacion_add_sub,Mutacion_add_sub-Bach2Frescobaldi
70,Bach,bwv0253.mid,<roll.guoroll.GuoRoll object at 0x7f14725dca60>,"[-0.6771048, -0.15007068, -0.12760165, 0.50512...","[-0.9170938, -0.69383276, -1.194445, 1.6231658...","[-0.70293194, -0.3269844, -0.22040199, -1.6397...",<roll.guoroll.GuoRoll object at 0x7f146b9d1910>
29,Bach,bwv0254.mid,<roll.guoroll.GuoRoll object at 0x7f1471ab8070>,"[-2.185468, 1.9231098, 1.0495294, 5.617977, 2....","[-2.425457, 1.3793477, -0.017313957, 6.736017,...","[-2.2112951, 1.746196, 0.9567291, 3.4731061, 1...",<roll.guoroll.GuoRoll object at 0x7f146b9e9610>
91,Bach,bwv0255.mid,<roll.guoroll.GuoRoll object at 0x7f1471a8de80>,"[-0.8813409, -0.5599785, -1.5920982, 3.2512496...","[-1.12133, -1.1037406, -2.6589417, 4.36929, 1....","[-0.9071681, -0.7368922, -1.6848986, 1.1063786...",<roll.guoroll.GuoRoll object at 0x7f146b978c40>
98,Bach,bwv0256.mid,<roll.guoroll.GuoRoll object at 0x7f1471a6ba00>,"[0.45691258, -0.8111836, -1.684641, 0.9001678,...","[0.21692352, -1.3549457, -2.7514844, 2.018208,...","[0.43108544, -0.9880973, -1.7774414, -1.244703...",<roll.guoroll.GuoRoll object at 0x7f146b996070>
47,Bach,bwv0258.mid,<roll.guoroll.GuoRoll object at 0x7f1471988ee0>,"[-0.6578047, 2.4769223, 1.2892462, -1.7592773,...","[-0.89779377, 1.9331602, 0.22240281, -0.641237...","[-0.6836319, 2.3000085, 1.1964458, -3.9041483,...",<roll.guoroll.GuoRoll object at 0x7f146b936ac0>


In [ ]:
h = df_transferred.head(1)
for f in h.iterrows():
    for c in f:
        print(c)
    print("-------------\nSolo el autor:", c["Autor"])

In [ ]:
for f in os.listdir(f"{data_path}embeddings/brmf_4b"):
    if "df_transferred" in f:
        df = load_pickle(f"{data_path}embeddings/brmf_4b/{f}")
        save_midis(df, verbose=True)